In [1]:
from requests import get
from bs4 import BeautifulSoup

In [2]:
url = "https://www.deakin.edu.au/study/find-a-course/courses-study-area"
soup = BeautifulSoup(get(url).content, "html.parser")

In [3]:
# this approach is very unstructured and skipped for now
# soup.body.findAll("div",  {"class": "module__content-panel module__content-panel-rebrand", "data-section": 'what area do you want to study in?'})

In [5]:
url  = "https://www.deakin.edu.au/trimester-one"
soup = BeautifulSoup(get(url).content, "html.parser")

In [6]:
sou.find('h3', string='Locations').parent.parent.find('ul')

NameError: name 'sou' is not defined

In [27]:
sou.find('picture').source.get('srcset')

'https://www.deakin.edu.au/__data/assets/image/0009/2508615/35282_graduate-diploma-emergency-nursing_landscape.jpg'

In [34]:
from os.path import exists

rows = soup.body.find("div", {"class":'module__content-panel--table--data-table'}).table.findAll('tr')
for row in rows[1:]:
    cname = row.td.a.text.replace('/', ' ')
    fname = f'course-bg/{cname}.jpg'
    if exists(fname):
        continue
    else: 
        sou = BeautifulSoup(get('https:' + row.td.a['href']).content, "html.parser")
        temp = get(sou.find('picture').source.get('srcset')).content
        with open(fname, 'wb') as handler:
            handler.write(temp)

In [26]:
# code taken from chatgpt : prompt
import re


def format_string(text):
    # Remove HTML tags
    clean_text = re.sub(r'<.*?>', '', text)
    
    # Remove extra whitespace
    clean_text = re.sub(r'\s+', ' ', clean_text)
    
    # Split the text into individual sentences
    sentences = clean_text.strip().split('. ')
    
    # Remove empty sentences & Join the sentences into a formatted string
    formatted_text = '. '.join([sentence for sentence in sentences if sentence])
    
    return formatted_text


def extract_data(_sou, datakey):
    temp = [i.text for i in _sou.body.findAll('div', {'data-section': datakey})]
    if len(temp):
        return format_string(temp[0].strip().replace('\n', ' '))
    else:
        return ''


rows = soup.body.find("div", {"class":'module__content-panel--table--data-table'}).table.findAll('tr')
data = {'course_name': [], 'url': [], 'overview': [], 'requirements': [], 'fee': [], 'careers': []}
for row in rows[1:]:
    data['course_name'].append(row.td.a.text)
    data['url'].append(row.td.a['href'])
    
    sou = BeautifulSoup(get('https:' + row.td.a['href']).content, "html.parser")
    par = sou.find('h3', string='Locations').parent.parent
    locations = []
    if par.find('ul'):
        [locations.append(i.get_text()) for i in par.find_all('li')]
    else:
        [locations.append(i.get_text()) for i in par.find_all('a')]
    print(locations)
    
    overview = extract_data(sou, 'course overview')
    requirements = extract_data(sou, 'entry requirements')
    fee = extract_data(sou, 'fees and scholarships')
    careers = extract_data(sou, 'careers')
    
    data['overview'].append(overview)
    data['requirements'].append(requirements)
    data['fee'].append(fee)
    data['careers'].append(careers)
    
    
    
    break

    

['Online']


In [7]:
from pandas import DataFrame


course_data = DataFrame(data)
course_data

,course_name,url,overview,requirements,fee,careers
0,Bachelor of Nursing (Honours),//www.deakin.edu.au/course/bachelor-nursing-ho...,"Course overview Focusing on research, the hono...",Entry requirements Higher education study If y...,Fees and scholarships Fee information Estimate...,Careers Career outcomes After successfully com...
1,Graduate Certificate of Intensive Care Nursing,//www.deakin.edu.au/course/graduate-certificat...,Course overview With job growth of 13.9% proje...,Entry requirements Entry information Bachelor ...,Fees and scholarships Fee information Estimate...,Careers Want a degree that’s more than just a ...
2,Graduate Certificate of Critical Care Nursing,//www.deakin.edu.au/course/graduate-certificat...,Course overview Critical care nurses work in s...,Entry requirements Entry information Bachelor ...,Fees and scholarships Fee information Estimate...,Careers Want a degree that’s more than just a ...
3,Graduate Diploma of Emergency Nursing,//www.deakin.edu.au/course/graduate-diploma-em...,Course overview Nurses working in emergency de...,Entry requirements Entry information Bachelor ...,Fees and scholarships Fee information Estimate...,Careers Want a degree that’s more than just a ...
4,Bachelor of Science (Honours),//www.deakin.edu.au/course/bachelor-science-ho...,Course overview Deakin’s Bachelor of Science (...,Entry requirements Entry information Entry wil...,Fees and scholarships Fee information Estimate...,Careers Career outcomes The Bachelor of Scienc...
...,...,...,...,...,...,...
285,Graduate Certificate of Teaching English to Sp...,//www.deakin.edu.au/course/graduate-certificat...,Course overview Become qualified to teach Engl...,Entry requirements Entry information Entry wil...,Fees and scholarships Fee information Estimate...,Careers Career outcomes There is a high demand...
286,Master of Social Work,//www.deakin.edu.au/course/master-social-work,"Course overview With a Master of Social Work, ...",Entry requirements Entry information Entry wil...,Fees and scholarships Fee information Estimate...,Careers Career outcomes The two year Master of...
287,Graduate Diploma of Psychology (Advanced),//www.deakin.edu.au/course/graduate-diploma-ps...,Course overview If you are looking to build on...,Entry requirements Higher education study If y...,Fees and scholarships Fee information Estimate...,Careers Career outcomes This course is a pathw...
288,Bachelor of Psychological Science (Honours),//www.deakin.edu.au/course/bachelor-psychologi...,Course overview Gain an insight into why peopl...,Entry requirements Higher education study If y...,Fees and scholarships Fee information Estimate...,Careers Want a degree that’s more than just a ...


In [8]:
course_data.to_csv('course-data.csv', index=False)

In [1]:
from pandas import read_csv


course_data = read_csv('course-data.csv')
course_data['overview'] = course_data.overview.str.replace('Course overview ', '').str.replace(' Read More', '')
course_data.to_csv('course-data.csv', index=False)

In [ ]:
from pandas import read_csv


course_data = read_csv('course-data.csv')
course_data['overview']
course_data.to_csv('course-data.csv', index=False)

In [2]:
from pandas import read_csv


course_data = read_csv('course-data.csv')
course_data[['course_name', 'overview', 'requirements']].to_csv('course-data-small.csv', index=False)